
# 📓 阶段 1 · 1.1 专用 Notebook —— 预测 vs 因果（医学入门）

本 Notebook 配套你的 **1.1 因果推断与预测的区别（入门者强化版）**：  
- 通过**同一份数据**，展示“预测”和“因果”得到的**不同结论**；  
- 提供可复用的**对照流程清单**；  
- 输出你需要报告/检查的关键指标（AUC、SMD、PS 重叠、ATE）。



## 1. 我们到底在问什么？

- **预测（Predictive）**：给定特征，预测结局 \(P(Y\mid X)\)。  
- **因果（Causal）**：如果我们**干预**治疗 \(X\)，结局会如何变化 \(P(Y\mid do(X))\)，或 \(\mathbb{E}[Y(1)-Y(0)]\)。

> 预测帮你“看风险”；因果帮你“改结果”。



## 2. 最小对照流程（各 5 步）

**做预测**（例：ICU 28 天死亡风险）：  
1) 明确目标（预测概率/风险分层）  
2) 选特征（对预测有用即可，避免信息泄露）  
3) 划分训练/验证/测试（此处为演示，先不划分）  
4) 训练评估（AUC、校准）  
5) 监控与再训练计划  

**做因果**（例：早期抗生素是否降低死亡）：  
1) 定义治疗/结局/时间线（协变量来自**治疗前**）  
2) 识别混杂并画 DAG（此处用模拟数据代替）  
3) 估 PS，检查**平衡**（SMD）与**重叠**（PS 分布）  
4) 用 IPTW/PSM/DR 估 ATE/HR/CATE  
5) 做敏感性分析与亚组分析（演示中先不展开）



## 3. 同一数据，不同问题：模拟数据演示

- 设定：重症程度（SOFA）越高，**越可能被治疗**（产生**混杂**）；实际真相是**治疗可降低死亡**。  
- 结果：
  - **预测模型**可能会把“被治疗”与“死亡更高”联系起来（因为重症更常被治疗），**不是因果**；  
  - **因果校正**（IPTW）后可更接近“治疗降低死亡”的真相。


In [ ]:

import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

np.random.seed(42)
n = 3000
age = np.random.normal(70, 10, n)            # 混杂：年龄越大越重
sofa = np.clip(np.random.normal(5 + 0.1*(age-70), 2, n), 0, None)

# 治疗分配：越重越容易用药（混杂来源）
logit_t = -2 + 0.4*sofa + 0.01*(age-70)
p_treat = 1/(1+np.exp(-logit_t))
treat = (np.random.rand(n) < p_treat).astype(int)

# 真因果：用药降低死亡，但重症本身提升死亡风险
logit_y = -3 + 0.5*sofa + 0.02*(age-70) - 0.6*treat
p_dead = 1/(1+np.exp(-logit_y))
y = (np.random.rand(n) < p_dead).astype(int)

df = pd.DataFrame({'age':age,'sofa':sofa,'treat':treat,'dead':y})
df.head()



### 3.1 预测问题：给定特征，预测死亡概率

- 我们用 `treat, age, sofa` 作为特征来做**死亡预测**（逻辑回归）。  
- 评价 AUC，并查看 `treat` 的回归系数（注意：**系数方向不是因果方向**）。


In [ ]:

X_pred = df[['treat','age','sofa']]
m_pred = LogisticRegression(max_iter=1000).fit(X_pred, df['dead'])
auc = roc_auc_score(df['dead'], m_pred.predict_proba(X_pred)[:,1])
coef_treat = m_pred.coef_[0][0]

print("预测 AUC:", round(auc, 3))
print("预测模型中 treat 系数（仅相关方向，不代表因果）:", round(coef_treat, 3))



### 3.2 因果问题（准备）：PS 重叠与基线平衡

- 只用**治疗前协变量**（这里用 `age, sofa`）来估计**倾向评分**（PS = 接受治疗的概率）。  
- 看看两组 PS 分布有没有**重叠**（positivity）。


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 倾向评分（只用治疗前协变量）
scaler = StandardScaler()
X_ps = scaler.fit_transform(df[['age','sofa']])
ps_model = LogisticRegression(max_iter=1000).fit(X_ps, df['treat'])
ps = ps_model.predict_proba(X_ps)[:,1]
df['ps'] = ps

plt.figure()
plt.hist(df.loc[df.treat==1, 'ps'], bins=30, alpha=0.6, label='Treatment')
plt.hist(df.loc[df.treat==0, 'ps'], bins=30, alpha=0.6, label='Control')
plt.xlabel('Propensity score'); plt.ylabel('Count'); plt.legend(); plt.title('PS overlap check')
plt.show()

# 简单 SMD（连续变量）
def smd_cont(x_t, x_c):
    mu_t, mu_c = np.nanmean(x_t), np.nanmean(x_c)
    sd_pool = np.sqrt((np.nanvar(x_t, ddof=1)+np.nanvar(x_c, ddof=1))/2)
    return (mu_t - mu_c) / (sd_pool + 1e-12)

tmask, cmask = df['treat']==1, df['treat']==0
print("SMD(age):", round(smd_cont(df.loc[tmask,'age'], df.loc[cmask,'age']),3))
print("SMD(sofa):", round(smd_cont(df.loc[tmask,'sofa'], df.loc[cmask,'sofa']),3))



### 3.3 因果估计：用 IPTW 粗估 ATE

- 稳定化权重：Treatment: \(w=1/PS\)，Control: \(w=1/(1-PS)\)。  
- 计算**加权后的**治疗组/对照组死亡率，再取差异（ATE）。  
- 结果若为**负值**，表示治疗可**降低**死亡率（与模拟真相一致）。


In [ ]:

import numpy as np

w = np.where(df['treat']==1, 1/df['ps'], 1/(1-df['ps']))
ate_iptw = (w[df.treat==1]*df.dead[df.treat==1]).sum()/w[df.treat==1].sum() \
         - (w[df.treat==0]*df.dead[df.treat==0]).sum()/w[df.treat==0].sum()

print("IPTW 估计的 ATE（>0=增加死亡，<0=降低死亡）:", round(ate_iptw, 4))



## 4. 关键结论与报告要点

- **预测**：报 AUC/校准等预测指标；**不要**把系数当成因果解释。  
- **因果**：报 ATE/RR/HR；提供**平衡性（SMD）**、**重叠性（PS 分布）**与**稳健性**说明。  
- 同一份数据：预测可能显示“治疗↔更高死亡”的**相关**；校正混杂后，因果估计可显示“治疗降低死亡”的**效果**。

> 建议：把本 Notebook 存为你团队的“对照演示”，帮助新人快速理解“预测 vs 因果”的核心差异。
